In [27]:
!pip install opencv-python -q

In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import random
import seaborn as sns
import matplotlib.pyplot as plt

In [44]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/breast-histopathology-images")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/breast-histopathology-images


In [45]:
directorios = os.listdir('/kaggle/input/breast-histopathology-images/IDC_regular_ps50_idx5')

In [46]:
rutas_imgs_no_cancer = []
rutas_imgs_cancer = []
for directorio in directorios:
  for subdirectorio in os.listdir(f'{path}/{directorio}'):
    if subdirectorio == '0':
      for imagen in os.listdir(f'{path}/{directorio}/{subdirectorio}'):
        rutas_imgs_no_cancer.append(f'{path}/{directorio}/{subdirectorio}/{imagen}')
    else:
      for imagen in os.listdir(f'{path}/{directorio}/{subdirectorio}'):
        rutas_imgs_cancer.append(f'{path}/{directorio}/{subdirectorio}/{imagen}')

In [47]:
print(rutas_imgs_no_cancer[0].split('/')[6])

10295_idx5_x1351_y1101_class0.png


In [48]:
for imagen in rutas_imgs_no_cancer:
  if imagen.split('/')[6][-5] != '0':
    print(imagen.split('/')[5])

In [49]:
for imagen in rutas_imgs_cancer:
  if imagen.split('/')[6][-5] != '1':
    print(imagen.split('/')[5])

In [50]:
print(f'Muestras Positivas {len(rutas_imgs_cancer)}\nMuestras Negatias {len(rutas_imgs_no_cancer)} ')

Muestras Positivas 78786
Muestras Negatias 198738 


Las clases claramente estan desbalanceadas.
Algunas soluciones para dar equilibrio y que este problema no ocurra son:
- Disminuir la clase mayoritatia
- Aumentar la clase minoritaria mediante Data Augmentation

Una buena pregunta seria:
Nivelar las instancias de ambas clases para que esten equilibradas no seria de alguna manera alterar la 'realidad' o la naturaleza/proporcionalidad en la que verdaderamente se manfiesta la variable en la vida real.

Por ejemplo:
Si una enfermedad se manfiesta de tal que forma que por cada persona enferma existen dos personas sanas y el dataset que se pretende explotar tiene 2 millones de instancias de personas sanas y un millon de personas que padecen la enfermedad, esta realmente desbalanceado o simplemente atiende a la realidad?
Anotacion:
Quiza la solucion sea una especie de termino medio como que la proporcion sea 1.2-1.6 por ejemplo el rango es formulado de manera aleatoria.

Pensando un poco mas alla surge otra pregunta mas en el caso de que esto afecte es lo mismo rebajar la clase mayoritaria (submuestreo) que aumentar la minoritaria (sobremuestreo)

SUBMUESTREO
- PROS
  - Mas rapido/ Menos recursos
- CONTRAS
  - Se pueden perder muestras representativas

# Preparacion de las imagenes


Las medidas durante la preparacion de las imagenes son:
- Normalización.
- Redimensionar porque a pesar de que se indica que todas estan  50x50 no es asi.
- Se mantienen a color puesto que este aporta informacion util.

In [51]:
rutas_imgs_no_cancer = rutas_imgs_no_cancer[0:int(5000* 1.3)]
rutas_imgs_cancer = rutas_imgs_cancer[0:5000]

In [52]:
# CARGAR IMAGENES
imgs_cancer = [cv2.imread(ruta) for ruta in rutas_imgs_cancer]
imgs_no_cancer = [cv2.imread(ruta) for ruta in rutas_imgs_no_cancer]

In [53]:
for img in imgs_cancer:
  if img.shape != (50,50,3):
    print(img.shape)

In [54]:
for img in imgs_no_cancer:
  if img.shape != (50,50,3):
    print(img.shape)

(3, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(37, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)
(39, 50, 3)


In [38]:
# REDIMENSIONAR Y NORMALIZAR
MEDIDAS = (50,50)
imgs_cancer = [[cv2.resize(imagen, MEDIDAS, interpolation=cv2.INTER_LINEAR)/255, 1] for imagen in imgs_cancer]
imgs_no_cancer = [[cv2.resize(imagen, MEDIDAS, interpolation=cv2.INTER_LINEAR)/255, 0] for imagen in imgs_no_cancer]

# TRAIN TEST

In [39]:
# MEZCLAR LAS MUESTRAS
datos = imgs_cancer+imgs_no_cancer
random.shuffle(datos)

X = []
y = []
# SEPARAR EN IMAGENES Y ETIQUETAS
for imagen, etiqueta in datos:
  X.append(imagen)
  y.append(etiqueta)

X = np.array(X)
y = np.array(y)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=100303)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=100303)

# MODELO

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Flatten, Input, Activation


In [7]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False, input_shape=(50, 50, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
historial = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)


In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Matriz de Confusión')
plt.show()


In [ ]:
import pickle
model.save('modelo_cancer_de_mama.keras')
model.save('modelo_cancer_de_mama.h5')
with open('historial_modelo_cancermama.pkl', 'wb') as file:
    pickle.dump(historial, file)